# Chapter 4 搜索与排名
这一章开始，介绍的内容是全文搜索引擎，可以在大量文档中搜索一系列单词，并根据这些单词的相关程度对结果进行排名。 下面需要完成的任务有：
- 搜集文档（爬虫）  
- 为文档建立索引  
- 结果查询  
- 结果查询的优化

## 4.1 简单的爬虫程序  
爬虫能够方便的帮我们获得一系列的网页文档，它能根据一组待建索引的网页，根据这些网页内部的链接而找到其他页面，对这些页面又采用同样的方法。有点类似蜘蛛网一样，一层一层往外扩散。    
这里用到了`urllib2`和`bs4`（BeautifulSoup是旧版）两个库，


In [56]:
import re
import sqlite3 
import urllib2
from bs4 import *
from urlparse import urljoin


ignorewords = set(['the', 'to', 'of', ' and', 'a', 'in', 'is', 'it'])

class crawler:
    def __init__(self, dbname):
        self.con = sqlite3.connect(dbname)
    
    def __del__(self):
        self.con.close()
    
    def dbcommit(self):
        self.con.commit()
    
    def createindextables(self):  # 创建数据库表格和索引
        self.con.execute('create table urllist(url)')
        self.con.execute('create table wordlist(word)')
        self.con.execute('create table wordlocation(urlid,wordid,location)')
        self.con.execute('create table link(fromid integer,toid integer)')
        self.con.execute('create table linkwords(wordid,linkid)')
        self.con.execute('create index wordidx on wordlist(word)')
        self.con.execute('create index urlidx on urllist(url)')
        self.con.execute('create index wordurlidx on wordlocation(wordid)')
        self.con.execute('create index urltoidx on link(toid)')
        self.con.execute('create index urlfromidx on link(fromid)')
        self.dbcommit()
    
    def getentryid(self, table, field, value, createnew=True):
        cur = self.con.execute(
            "select rowid from %s where %s=%s" % (table, field, value)
            )
        res = cur.fetchone()
        if res == None:
            cur = self.con.execute(
                "insert into %s (%s) value ('%s')" % (table, field, value)
            )
            return cur.lastrowid
        else:
            return res[0]
    
    def addtoindex(self, url, soup):
        if self.isindexed(url):
            return
        print 'Indexing %s' % url
        
        # 获取每个单词
        text = self.gettextonly(soup)
        words = separatewords(text)
        
        # 得到URL的id
        urlid = self.getentryid('urllist', 'url', url)
        
        # 将单词与url进行关联
        for i in range(len(words)):
            word = words[i]
            if word in ignorewords:
                continue
            wordid = self.getentryid('wordlist', 'word', word)
            self.con.execute('insert into wordlocation(urlid, wordid, location) \
                values (%d, %d, %d)' % (urlid, wordid, i))
        
    def gettextonly(self, soup):
        v = soup.string
        if v == None:
            c = soup.contents
            resulttext = ''
            for t in c:
                subtext = self.gettextonly(t)
                resulttext += subtext + '\n'
            return resulttext
        else:
            return v.strip()
    
    def separatewords(self, text):   # 将文本划分成单词的程序
        splitter = re.compile(r'\W*')
        return [s.lower() for s in splitter.split(text) if s != ""]
    
    def isindexed(self, url):
        u = self.con.execute(
            "select rowid from urllist where url='%s'" % url).fetchone()
        if u != None:
            v = self.con.execute(
                "select * from wordlocation where urlid=%s" % u[0]).fetchone()
            if v != None:
                return True
        return False
    
    def addlinkref(self, urlFrom, urlTo, linkText):
        pass
    
    def crawl(self,pages, depth=2):
        for i in range(depth):    # 爬取的循环为2，避免过多的深入，造成可能的无限循环
            newpages = set()    # 建立一个集合来存储，可以去除重复的url
            for page in pages:
                try:
                    c=urllib2.urlopen(page)
                except:
                    print "Could not open %s" % page
                    continue 
                soup = BeautifulSoup(c.read())
                self.addtoindex(page, soup)
                
                links = soup('a')  # html中链接以<a>形式的tag进行标记
                for link in links:
                    if ('href' in dict(link.attrs)):
                        url=urljoin(page, link['href'])
                        #print url
                        if url.find("'") != -1:   # 当url中出现' 时不需要添加
                            continue
                        url = url.split('#')[0]  # 去除网页位置信息  
                        if url[0:4] == 'http' and not self.isindexed(url):
                            newpages.add(url)
                        linkText = self.gettextonly(link)
                        self.addlinkref(page, url, linkText)
            self.dbcommit()
        #print "newpages: ", newpages
        pages = newpages

        

In [57]:
pagelist = [u'http://baike.baidu.com/item/%E6%9E%B8%E6%9D%9E']
cra = crawler("searchindex.db")
cra.crawl(pagelist)


OperationalError: disk I/O error

## 4.2 建立索引  
上面已经能够由输入的网址而获得一系列相关的网站信息，这些信息保存于`newpages`中。之后需要对获得的网页建立索引，有了索引能够更方便的帮助我们查询包括单词的文档、单词所在位置等信息。对于网页中的内容，需要做这下面几步：
- 过滤掉了所有的标点符号  
- 忽略所有非文本元素  
- 对文本进行分词  

索引使用的数据库为SQLite数据库，简单方便，一般python都自带相关的模块（这里使用`sqlite3`与原书不一致）来处理数据库连接。继续对上面的脚本进行修改，添加数据库操作相关的代码。 一共需要添加5个表格，同时各自建立了索引以方便检索：
1. link：保存2个链接，指明由哪个链接到哪个链接的关系  
2. urllist：保存索引过的url列表  
3. wordlocation：保存单词在文档中的位置信息  
4. linkwords：表明单词存在于哪个链接中   
5. wordlist：保存的是单词列表

In [44]:
import sqlite3